In [ ]:
# Import essential packages
from bs4 import BeautifulSoup
import pandas

# Clean and store data
def process_with_pandas(soup_obj: BeautifulSoup, store: list):
    
    items = soup_obj.find_all(name='div', attrs={'class': 'caption'})

    for item in items:
        if item.find(class_='product-rating') is not None:
            rating = float(item.find(class_='product-rating').get_text())
        else:
            rating = 0.0  # refers to no rating
        info_dict = {'name': item.h3.get_text(), 'html_link': item.h3.a.attrs['href'],
                     'price': item.find(class_="price").get_text(), 'rating': rating}

        # Navigate to the product page and extract the color information
        driver.get(info_dict['html_link'])
        sleep(2.5)
        product_soup = BeautifulSoup(driver.page_source, 'lxml')
        color_div = product_soup.find(name='div', attrs={'class': 'option-image'})
        if color_div is not None:
            color_links = color_div.find_all(name='data-v-99597412')
            colors = [link.get_text() for link in color_links]
            info_dict['colors'] = colors
        else:
            info_dict['colors'] = []

        store.append(info_dict)
        

def process_data(df):
    # process the price string to float value of price
    # for those having a discount, calculate discount percentage
    for i in range(0, len(df)):
        s = df['price'][i]
        if s[2] == 'p':
            df.loc[i, 'price'] = float(s[12:-2].replace(',', '.'))
            df.loc[i, 'discounted'] = 0.0
        else:
            df.loc[i, 'price'] = float(s.split(' €')[0].replace(',', '.'))
            if len(s.split(' €')) > 2:
                df.loc[i, 'discounted'] = 1 - float(s.split(' €')[0].replace(',', '.')) / float(
                    s.split(' €')[1].replace(',', '.'))
            else:
                df.loc[i, 'discounted'] = 0.0

df = df.round({'discounted': 2})

    # Count the number of colors available for each product
    df['num_colors'] = df['colors'].apply(lambda x: len(x))

    return df
